In [1]:
import pandas as pd
import datetime
import os

os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/stores_in_2_DMA_20190701'

In [2]:
latest_agg_sales=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2019-06-22.csv",dtype=str,usecols=['location_id','sales','week_end_date'])
latest_agg_sales['sales']=latest_agg_sales['sales'].astype(float)

latest_agg_sales=latest_agg_sales[latest_agg_sales['sales']>0]


In [3]:
week_end_dates_2019=[datetime.date(2019,6,22)+datetime.timedelta(days=7*x) for x in range(6)]
week_end_dates_2018=[x - datetime.timedelta(days=52*7) for x in week_end_dates_2019]

week_end_dates_2019=[str(x) for x in week_end_dates_2019]
week_end_dates_2018=[str(x) for x in week_end_dates_2018]


In [4]:
latest_agg_sales=latest_agg_sales[latest_agg_sales['week_end_date'].isin(week_end_dates_2019+week_end_dates_2018)]

In [5]:
store_list_201906=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190601-134302-813.txt",dtype=str,usecols=['location_id','zip_cd'],sep="|")
store_list_201907=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190701-134908-815.txt",dtype=str,usecols=['location_id','zip_cd'],sep="|")
store_list_201807=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20180703.txt",dtype=str,usecols=['location_id','zip_cd'],sep="|")

all_store_list=[store_list_201906,store_list_201907,store_list_201807]

all_store_df=pd.DataFrame()
for df in all_store_list:
    df['zip_cd']=df['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
    all_store_df=all_store_df.append(df).drop_duplicates()

In [6]:
print(all_store_df.shape)
print(all_store_df['location_id'].nunique())

all_store_df[all_store_df['location_id']=="4604"]

(1470, 2)
1469


,location_id,zip_cd
1136,4604,85283
1002,4604,85282


In [7]:
Nielsen_DMA=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",skiprows=1,usecols=[0,2],dtype=str)
Nielsen_DMA=Nielsen_DMA.drop_duplicates()
Nielsen_DMA.columns=['zip_cd','DMA']
# sorted(Nielsen_DMA['DMA'].unique().tolist())

test_DMA_list=['CHARLOTTE','BUFFALO']
Nielsen_DMA=Nielsen_DMA[Nielsen_DMA['DMA'].isin(test_DMA_list)].drop_duplicates()

In [8]:
all_store_df=pd.merge(all_store_df,Nielsen_DMA,on="zip_cd",how="left")
all_store_df=all_store_df[pd.notnull(all_store_df['DMA'])]

In [9]:
"4604" in all_store_df['location_id'].unique().tolist()

False

In [10]:
latest_agg_sales=pd.merge(latest_agg_sales,all_store_df,on="location_id",how="left")
latest_agg_sales=latest_agg_sales[pd.notnull(latest_agg_sales['DMA'])]

In [12]:
latest_agg_sales=latest_agg_sales.sort_values(["DMA","location_id","week_end_date"])

In [11]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/stores_in_2_DMA_20190701'

In [16]:
summary_store_weeks=latest_agg_sales.groupby(['DMA',"location_id"])['week_end_date'].count().to_frame().reset_index().sort_values(["DMA","location_id","week_end_date"])

In [17]:
writer=pd.ExcelWriter('/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/stores_in_2_DMA_20190701/BL_stores_in_Charlotte_Buffalo_JL_'+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
latest_agg_sales.to_excel(writer,"sales_by_store_week",index=False)
summary_store_weeks.to_excel(writer,"summary_count_of_weeks",index=False)

writer.save()